### Final Project. Due Friday, Dec 15 by midnight.

---

Several of the questions build on each other. It might be beneficial to write some views for those so you don't end up duplicating the same CTEs/subqueries.

Thanks for a great semester!

---

In [1]:
import store 
import pandas as pd
# To force python to reload the module if already loaded
from importlib import reload
reload(store);
import sqlite3
# You'll find this library on blackboard
import dbga
# python libaries specific to making your submission file
from collections import defaultdict as dd
import json

In [2]:
dbga.__version__

'1.0.4'

In [3]:
conn = sqlite3.connect('/home/jovyan/Databases/04_Building/store.db')
curs = conn.cursor()

In [4]:
db = store.StoreDB('/home/jovyan/Databases/04_Building/store.db')
db.connect()
db.conn.execute("PRAGMA foreign_keys = ON;")
db.run_query("SELECT * FROM sqlite_master;")

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,tState,tState,2,CREATE TABLE tState (\n sta...
2,index,sqlite_autoindex_tState_1,tState,4,None
3,table,tZip,tZip,5,CREATE TABLE tZip (\n zip T...
4,index,sqlite_autoindex_tZip_1,tZip,6,None
5,table,tCust,tCust,7,CREATE TABLE tCust (\n cust...
6,table,tOrder,tOrder,8,CREATE TABLE tOrder (\n ord...
7,table,tProd,tProd,9,CREATE TABLE tProd (\n prod...
8,table,tOrderDetail,tOrderDetail,10,CREATE TABLE tOrderDetail (\n ...
9,index,sqlite_autoindex_tOrderDetail_1,tOrderDetail,11,None


In [5]:
db.drop_all_tables(are_you_sure=True)

In [6]:
db.build_tables()

Table tState built successfully.
Table tZip built successfully.
Table tCust built successfully.
Table tOrder built successfully.
Table tProd built successfully.
Table tOrderDetail built successfully.


In [7]:
db.load_lookup_tables()

Loading lookup tables...
State data loaded:         state state_id
0     Alabama       AL
1      Alaska       AK
2     Arizona       AZ
3    Arkansas       AR
4  California       CA
Data loaded into table.
State data loaded into tState table.
Product data loaded:    prod_id prod_name  unit_price
0      300    Washer        0.10
1      301      Nail        0.25
2      302       Nut        0.25
3      303     Screw        0.50
4      304      Bolt        1.00
Data loaded into table.
Product data loaded into tProd table.
Zip data loaded:    zip       city state_id
0  601   Adjuntas       PR
1  602     Aguada       PR
2  603  Aguadilla       PR
3  606    Maricao       PR
4  610     Anasco       PR
Data loaded into table.
Zip data loaded into tZip table.
Lookup tables loaded.


In [8]:
db.load_new_data('/home/jovyan/Databases/sales_folder', '/home/jovyan/Databases/processed_data')

File Sales_201901.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201902.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201903.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201904.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201905.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201906.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201907.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201908.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201909.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201910.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201911.csv processed and moved to /home/jovyan/Databases/processed_data
File Sales_201912.csv processed and moved to /home/jovyan/Databases/processed_data
File

In [9]:
my_sql = dd(dict)

In [10]:
my_sql['setup'] = []

In [11]:
pd.read_sql("SELECT * FROM tOrder;", conn)

,order_id,cust_id,date
0,1,1,2019-01-01
1,2,2,2019-01-02
2,3,3,2019-01-02
3,4,4,2019-01-02
4,5,5,2019-01-02
...,...,...,...
2922,2923,75,2021-10-29
2923,2924,23,2021-10-29
2924,2925,222,2021-10-30
2925,2926,18,2021-10-31


In [12]:

my_sql['setup'] = []

# I do need a view for one of the questions
sql = "DROP VIEW IF EXISTS StateSalesView;"
my_sql['setup'].append(sql)
curs.execute(sql)

sql = '''
CREATE VIEW StateSalesView AS
WITH StateSales AS (
    SELECT tZip.state_id,
        IFNULL(SUM(tOrderDetail.qty * tProd.unit_price), 0) AS total_sales
    FROM tZip
        JOIN tCust USING (zip)
        JOIN tOrder USING (cust_id)
        JOIN tOrderDetail USING (order_id)
        JOIN tProd USING (prod_id)
    WHERE strftime('%Y-%m', tOrder.date) = '2019-09'
    GROUP BY tZip.state_id
)
SELECT s.state_id,
    IFNULL(ss.total_sales, 0) AS total_sales
FROM (
    SELECT DISTINCT state_id FROM tZip
) s
LEFT JOIN StateSales ss USING (state_id)
ORDER BY s.state_id;
'''
my_sql['setup'].append(sql)
curs.execute(sql)


curs.execute("DROP VIEW IF EXISTS vOrder_DatesParsed;")
sql = """
CREATE VIEW vOrder_DatesParsed as
SELECT *, SUBSTR(date, 1, 4) as year,
          SUBSTR(date, 6, 2) as month,
          SUBSTR(date, 9, 2) as day,
          SUBSTR(date, 1, 7) as year_month
FROM tOrder
;"""
curs.execute(sql)

---
1) Generate a summary, by month and year of how our store is performing.

Have your query return the following:
 - year_month (as a single column, like YYYY-MM)
 - Sales: total sales for the month (i.e., sum of qty * unit price)
 - NumOrders: number of orders placed for the month
 - NumCust: number of _distinct_ customers who made a purchase (i.e. only count the customer at most once per month)
 - OrdersPerCust: average number of orders per customer (i.e. NumOrders/NumCust)
 - SalesPerCust: average sales per customer (i.e. Sales/NumCust)
 - SalesPerOrder: average sales per order (i.e. Sales/NumOrders)

Sort the results should by year_month, in ascending order.

_Hint: Watch out for integer division!_

DONE

In [13]:
sql1 = '''
WITH TotalsPerMonth AS (
    SELECT
        strftime('%Y-%m', date) AS year_month,
        sum(qty * unit_price) AS TotalSales,
        count(DISTINCT order_id) AS NumOrders,
        count(DISTINCT cust_id) AS NumCust
    FROM tOrder
    JOIN tOrderDetail USING(order_id)
    JOIN tProd USING(prod_id)
    GROUP BY year_month
)
SELECT
    year_month,
    sum(TotalSales) AS Sales,
    sum(NumOrders) AS NumOrders,
    sum(NumCust) AS NumCust,
    AVG(NumOrders * 1.0 / NumCust) AS OrdersPerCust,
    AVG(TotalSales * 1.0 / NumCust) AS SalesPerCust,
    AVG(TotalSales * 1.0 / NumOrders) AS SalesPerOrder
FROM TotalsPerMonth
GROUP BY year_month
ORDER BY year_month ASC;
'''

pd.read_sql(sql1, conn) 

,year_month,Sales,NumOrders,NumCust,OrdersPerCust,SalesPerCust,SalesPerOrder
0,2019-01,68464.61,91,85,1.070588,805.466000,752.358352
1,2019-02,55560.32,80,73,1.095890,761.100274,694.504000
2,2019-03,19191.68,51,49,1.040816,391.666939,376.307451
3,2019-04,20912.07,48,46,1.043478,454.610217,435.668125
4,2019-05,11973.34,50,46,1.086957,260.290000,239.466800
5,2019-06,13737.30,43,41,1.048780,335.056098,319.472093
6,2019-07,22095.05,45,41,1.097561,538.903659,491.001111
7,2019-08,15675.05,51,49,1.040816,319.898980,307.353922
8,2019-09,9360.38,40,39,1.025641,240.009744,234.009500
9,2019-10,48411.35,58,51,1.137255,949.242157,834.678448


In [14]:
my_sql['q1']['sql'] = sql1

---
2) In which month did we have the lowest total sales?

Return one record with:
- year_month (as a single column, like YYYY-MM)
- sales (sum of qty * unit_price)

DONE

In [15]:
sql2 = '''
SELECT
    strftime('%Y-%m', date) AS year_month,
    SUM(tOrderDetail.qty * tProd.unit_price) AS sales
FROM
    tOrder
    JOIN tOrderDetail USING (order_id)
    JOIN tProd USING (prod_id)
    JOIN tCust USING (cust_id)
GROUP BY year_month
ORDER BY sales ASC
LIMIT 1
;'''
pd.read_sql(sql2, conn)  

,year_month,sales
0,2019-09,9360.38


In [16]:
my_sql['q2']['sql'] = sql2

---

3. In the month determined from the previous question, generate a list of our total sales by state. Make sure that all states are included, even if they have no sales (50 states + PR and DC = 52 total records). For states having 0 sales, return 0, not NULL or NaN.

Return:

- state_id
- Total sales for the month in question

Order the results by the state_id, ascending

Done

In [17]:
sql3 = '''
SELECT
    s.state_id,
    IFNULL(ss.total_sales, 0) AS total_sales
FROM (
    SELECT DISTINCT state_id FROM tZip
) s
LEFT JOIN StateSalesView ss USING (state_id)
ORDER BY s.state_id;
'''
pd.read_sql(sql3, conn)  

,state_id,total_sales
0,AK,0.00
1,AL,28.00
2,AR,0.00
3,AZ,0.00
4,CA,557.00
5,CO,0.00
6,CT,0.00
7,DC,0.00
8,DE,0.00
9,FL,414.00


In [18]:
my_sql['q3']['sql'] = sql3

---

4. For the list of states above that had 0 sales, generate a list of all the customers in those states, along with how much they have bought from us since then.

Return:
- cust_id
- first
- last
- addr
- city
- st
- zip
- the customer's total sales from all months after the month from question 4

Order the results by cust_id, ascending

Done

In [19]:
sql4 = '''
WITH CustomerSales AS (
    SELECT
        tCust.cust_id,
        tCust.first,
        tCust.last,
        tCust.addr,
        tZip.city,
        tZip.state_id AS st,
        tCust.zip,
        IFNULL(SUM(tOrderDetail.qty * tProd.unit_price), 0) AS total_sales
    FROM
        tCust
        JOIN tZip USING (zip)
        LEFT JOIN tOrder USING (cust_id)
        LEFT JOIN tOrderDetail USING (order_id)
        LEFT JOIN tProd USING (prod_id)
    WHERE
        strftime('%Y-%m', IFNULL(tOrder.date, '2019-09')) >= '2019-09'
    GROUP BY
        tCust.cust_id
)
SELECT
    cs.cust_id,
    cs.first,
    cs.last,
    cs.addr,
    cs.city,
    cs.st,
    cs.zip,
    cs.total_sales AS customer_total_sales
FROM
    CustomerSales cs
    JOIN StateSalesView ssv ON cs.st = ssv.state_id
WHERE
    ssv.total_sales = 0
ORDER BY
    cs.cust_id ASC;

    '''
pd.read_sql(sql4, conn)  

,cust_id,first,last,addr,city,st,zip,customer_total_sales
0,3,Dodonna,Garza,3639 Briarwood Court,Tarzan,TX,79783,364.90
1,6,Motti,Hunt,1919 Smith Street,Hamden,CT,6514,5465.67
2,7,Mace Windu,Woodward,5655 Jefferson Court,Milan,NM,87021,9422.33
3,11,Senator Ask Aak,Chen,2027 Pheasant Run,Wayne,NJ,7470,11028.03
4,12,Jabba,Oneill,9371 Walnut Street,Swampscott,MA,1907,9848.63
...,...,...,...,...,...,...,...,...
113,297,Tey How,Rodriguez,9164 Madison Street,Brick,NJ,8723,4347.75
114,301,Rabe,Mitchell,8021 Heather Lane,Pindall,AR,72669,8828.98
115,302,Taun We,Miller,4011 Sycamore Drive,Yellow Jacket,CO,81335,11850.42
116,305,Dodonna,Francis,8070 Briarwood Court,Buckland,AK,99727,4878.76


In [20]:
my_sql['q4']['sql'] = sql4

---

5) Get a list of customers who did not purchase anything in the most recent month of data, along with their average sales for all months prior.

Return:

- customer id
- name, address, zip, city, st (abbreviation is fine)
- average sales for all months prior (divide by N-1, where N is the total number of months in the database)

Order the results with the largest average monthly sales on top.

DONE

In [21]:
sql5='''
WITH LatestMonth AS (
    SELECT MAX(strftime('%Y-%m', date)) AS latest_month
    FROM tOrder
),
CustomersWithoutPurchase AS (
    SELECT DISTINCT tCust.cust_id
    FROM tCust
    WHERE tCust.cust_id NOT IN (
        SELECT DISTINCT cust_id
        FROM tOrder
        WHERE strftime('%Y-%m', date) = (SELECT latest_month FROM LatestMonth)
    )
),
Months AS (
    SELECT COUNT(DISTINCT strftime('%Y-%m', date)) - 1 AS NumMonths
    FROM tOrder
),
CustomerSales AS (
    SELECT
        tCust.cust_id,
        tCust.first || ' ' || tCust.last AS name,
        tCust.addr,
        tZip.zip,
        tZip.city,
        tZip.state_id AS st,
        IFNULL(SUM(tOrderDetail.qty * tProd.unit_price), 0) AS total_sales,
        strftime('%Y-%m', IFNULL(tOrder.date, '0000-00-00')) AS year_month
    FROM
        tCust
        JOIN tZip USING (zip)
        LEFT JOIN tOrder USING (cust_id)
        LEFT JOIN tOrderDetail USING (order_id)
        LEFT JOIN tProd USING (prod_id)
    WHERE
        tCust.cust_id IN (SELECT cust_id FROM CustomersWithoutPurchase)
        AND (strftime('%Y-%m', IFNULL(tOrder.date, '0000-00-00')) < (SELECT latest_month FROM LatestMonth))
    GROUP BY
        tCust.cust_id, year_month
),
AverageSales AS (
    SELECT
        cs.cust_id,
        cs.name,
        cs.addr,
        cs.zip,
        cs.city,
        cs.st,
        MAX(cs.year_month) AS year_month,
        cs.total_sales AS customer_total_sales,
        m.NumMonths,
        CASE
            WHEN m.NumMonths > 0 THEN 1. * cs.total_sales / m.NumMonths
            ELSE 0  
        END AS avg_sales_prior
    FROM
        CustomerSales cs
        CROSS JOIN Months m
    GROUP BY
        cs.cust_id, cs.name, cs.addr, cs.zip, cs.city, cs.st, m.NumMonths
)

SELECT
    cust_id,
    name,
    addr,
    zip,
    city,
    st,
    year_month,
    avg_sales_prior
FROM
    AverageSales
ORDER BY
    avg_sales_prior DESC;
'''
pd.read_sql(sql5,conn)

,cust_id,name,addr,zip,city,st,year_month,avg_sales_prior
0,245,Lieutenant Mitaka Oneill,1110 2nd Street,2836,Kenyon,RI,2021-08,205.875455
1,188,On Medon Curtis,1971 Lake Street,74570,Stuart,OK,2021-09,187.818485
2,22,Rieekan Jimenez,5165 5th Street,43149,Rockbridge,OH,2021-08,178.892121
3,198,Lama Su Vincent,2114 4th Street,23866,Ivor,VA,2021-07,160.983636
4,202,Gold Leader Elliott,9326 Sycamore Street,25938,Victor,WV,2021-08,153.463939
...,...,...,...,...,...,...,...,...
178,153,Padme Todd,9619 Beechwood Drive,6615,Stratford,CT,2021-08,1.694848
179,48,Tey How Barker,6169 Beechwood Drive,717,Ponce,PR,2021-06,0.927576
180,113,Colonel Datoo Schmidt,8706 Devonshire Drive,94523,Pleasant Hill,CA,2020-09,0.453030
181,116,Bala-Tik Schmidt,3353 Front Street,6053,New Britain,CT,2021-09,0.143939


In [22]:
my_sql['q5']['sql'] = sql5

---
6) Are there any products we haven't sold at least 1 of each month?

If so, return:
 
- prod_id
- prod_desc
- unit_price
- year_month

Order the results by prod_id, year_month, both ascending

DONE

In [23]:
sql6 = '''
WITH MonthlyProductSales AS (
    SELECT
        p.prod_id,
        p.prod_name AS prod_desc,
        p.unit_price,
        strftime('%Y-%m', o.date) AS year_month
    FROM
        tProd p
        CROSS JOIN (SELECT DISTINCT strftime('%Y-%m', date) AS year_month FROM tOrder) months
        LEFT JOIN tOrderDetail od ON p.prod_id = od.prod_id
        LEFT JOIN tOrder o ON od.order_id = o.order_id AND strftime('%Y-%m', o.date) = months.year_month
    WHERE
        o.order_id IS NOT NULL
)
, ProductsSoldInEachMonth AS (
    SELECT
        prod_id,
        year_month
    FROM
        MonthlyProductSales
    GROUP BY
        prod_id, year_month
    HAVING
        COUNT(*) > 0
)
, AllProductsAndMonths AS (
    SELECT
        p.prod_id,
        p.prod_name AS prod_desc,
        p.unit_price,
        months.year_month
    FROM
        tProd p
        CROSS JOIN (SELECT DISTINCT strftime('%Y-%m', date) AS year_month FROM tOrder) months
)
SELECT
    apm.prod_id,
    apm.prod_desc,
    apm.unit_price,
    apm.year_month
FROM
    AllProductsAndMonths apm
    LEFT JOIN ProductsSoldInEachMonth ps ON apm.prod_id = ps.prod_id AND apm.year_month = ps.year_month
WHERE
    ps.prod_id IS NULL
ORDER BY
    apm.prod_id ASC,
    apm.year_month ASC;

'''
pd.read_sql(sql6, conn)


,prod_id,prod_desc,unit_price,year_month
0,300,Washer,0.10,2019-09
1,301,Nail,0.25,2019-09
2,305,Bradawl,1.99,2019-09
3,307,Sandpaper,3.00,2019-03
4,308,Screwdriver,3.00,2019-05
5,309,Chisel,4.99,2019-07
6,310,Scraper,7.99,2019-04
7,312,Plane,10.99,2019-07
8,313,Wrench,11.00,2019-06
9,314,Mallet,12.00,2019-06


In [24]:
my_sql['q6']['sql'] = sql6

---
7) What are our top 5 selling products (in terms of total dollars sold)?

Return:

- prod_id
- prod_desc
- unit_price
- total sales (i.e. sum of qty * unit price)

Order the results by total sales, descending

DONE

In [25]:
sql7 = '''
SELECT prod_id,prod_name AS prod_desc,unit_price,SUM(qty * unit_price) AS total_sales
FROM tProd JOIN tOrderDetail USING (prod_id)
GROUP BY prod_id,prod_name,unit_price
ORDER BY total_sales DESC
LIMIT 5;
'''
pd.read_sql(sql7, conn)

,prod_id,prod_desc,unit_price,total_sales
0,329,Chainsaw,499.99,943481.13
1,328,Workbench,300.00,633300.00
2,327,Ladder,80.00,154000.00
3,326,Drill,69.00,131652.00
4,325,Toolbox,50.00,100850.00


In [26]:
my_sql['q7']['sql'] = sql7

---

8) What month did we have our highest sales, and what was our top selling product that month? (All in terms of dollars).

Return:

- year_month
- prod_id
- prod_desc
- The total sales for that month, for all products
- The total sales for that month, for the top selling product only
- The total quantity for that month, for the top selling product only

In [27]:
sql8 = '''
WITH MonthlyTotalSales AS (
    SELECT
        strftime('%Y-%m', o.date) AS year_month,
        SUM(od.qty * p.unit_price) AS total_sales_for_all_products
    FROM
        tOrder o
        JOIN tOrderDetail od ON o.order_id = od.order_id
        JOIN tProd p ON od.prod_id = p.prod_id
    GROUP BY
        year_month
),
MonthlyProductSales AS (
    SELECT
        strftime('%Y-%m', o.date) AS year_month,
        p.prod_id,
        p.prod_name AS prod_desc,
        SUM(od.qty * p.unit_price) AS total_sales_for_product,
        SUM(od.qty) AS total_quantity_for_product
    FROM
        tOrder o
        JOIN tOrderDetail od ON o.order_id = od.order_id
        JOIN tProd p ON od.prod_id = p.prod_id
    GROUP BY
        year_month,
        p.prod_id,
        p.prod_name
),
TopSellingProductPerMonth AS (
    SELECT
        year_month,
        prod_id,
        prod_desc,
        total_sales_for_product,
        total_quantity_for_product,
        RANK() OVER (PARTITION BY year_month ORDER BY total_sales_for_product DESC) AS sales_rank
    FROM
        MonthlyProductSales
)
SELECT
    mts.year_month,
    mts.total_sales_for_all_products,
    ts.prod_id,
    ts.prod_desc,
    MAX(ts.total_sales_for_product) AS total_sales_for_top_product,
    MAX(ts.total_quantity_for_product) AS total_quantity_for_top_product
FROM
    MonthlyTotalSales mts
    JOIN TopSellingProductPerMonth ts ON mts.year_month = ts.year_month
WHERE
    ts.sales_rank = 1
GROUP BY
    mts.year_month, ts.prod_id, ts.prod_desc
ORDER BY total_quantity_for_top_product DESC
LIMIT 1;
'''
pd.read_sql(sql8,conn)

,year_month,total_sales_for_all_products,prod_id,prod_desc,total_sales_for_top_product,total_quantity_for_top_product
0,2021-10,251768.12,329,Chainsaw,86998.26,174


In [28]:
my_sql['q8']['sql'] = sql8

In [29]:
conn.close()

In [30]:
with open('final.data302', 'w') as f:
    json.dump(my_sql, f)

In [31]:
dbga.validate_submission_format(path_db = '/home/jovyan/Databases/04_Building/store.db',
                                path_submission = 'final.data302',
                                num_questions = 8)

Report saved to:  VALIDATION_final.txt


In [32]:
for key, value in my_sql.items():
    print('key: ', key)
    print('value: ', value)
    print('\n')

key:  setup
value:  ['DROP VIEW IF EXISTS StateSalesView;', "\nCREATE VIEW StateSalesView AS\nWITH StateSales AS (\n    SELECT tZip.state_id,\n        IFNULL(SUM(tOrderDetail.qty * tProd.unit_price), 0) AS total_sales\n    FROM tZip\n        JOIN tCust USING (zip)\n        JOIN tOrder USING (cust_id)\n        JOIN tOrderDetail USING (order_id)\n        JOIN tProd USING (prod_id)\n    WHERE strftime('%Y-%m', tOrder.date) = '2019-09'\n    GROUP BY tZip.state_id\n)\nSELECT s.state_id,\n    IFNULL(ss.total_sales, 0) AS total_sales\nFROM (\n    SELECT DISTINCT state_id FROM tZip\n) s\nLEFT JOIN StateSales ss USING (state_id)\nORDER BY s.state_id;\n"]


key:  q1
value:  {'sql': "\nWITH TotalsPerMonth AS (\n    SELECT\n        strftime('%Y-%m', date) AS year_month,\n        sum(qty * unit_price) AS TotalSales,\n        count(DISTINCT order_id) AS NumOrders,\n        count(DISTINCT cust_id) AS NumCust\n    FROM tOrder\n    JOIN tOrderDetail USING(order_id)\n    JOIN tProd USING(prod_id)\n    G